In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import joblib

from sklearn import set_config
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

In [2]:
def nan_handler(data):
    data_copy = data.copy()
    # Replace infinite values with NaN
    data_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
    for column in data_copy.columns:
        if data_copy[column].isnull().any():
            data_copy[column].fillna(data_copy[column].median(), inplace=True)
    return data_copy